# Retry Query Engine

In [1]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from llama_index.indices.vector_store.base import GPTVectorStoreIndex
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.query_engine.retry_query_engine import RetryQueryEngine
from llama_index.query_engine.retry_source_query_engine import RetrySourceQueryEngine
from llama_index.readers.file.base import SimpleDirectoryReader
from llama_index.evaluation.base import QueryResponseEvaluator
from typing import cast

uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()
index = GPTVectorStoreIndex.from_documents(uber_docs)
query = "How many drivers does Uber have?"

# Default query engine
default_query_engine = index.as_query_engine()
response = default_query_engine.query(query)
print(f"Default query engine response: {response}")

# # Query engine with retry
evaluator = QueryResponseEvaluator()
retry_query_engine = RetryQueryEngine(default_query_engine, evaluator)
retry_response = retry_query_engine.query(query)
print(f"Query engine with retry response: {retry_response}")

# retry_source_query_engine = RetrySourceQueryEngine(default_query_engine, evaluator)
# retry_source_response = retry_source_query_engine.query(query)
# print(f"Query engine with retry source response: {retry_source_response}")

Default query engine response: 
It is not possible to answer this question with the given context information.
Query engine with retry response: 
Here is a better answer.

It is not possible to answer this question with the given context information. However, it is mentioned that the 2016 Breach affected the personal data of approximately 600,000 Drivers.
